# ETA Model Development - LogiMetrics

## Estimated Time of Arrival Prediction using Machine Learning

This notebook develops and trains a Random Forest model for predicting delivery ETAs based on:
- Distance and route characteristics
- Time-based features (hour, day, traffic patterns)
- Vehicle and shipment attributes
- Historical performance data

**Author:** LogiMetrics ML Team  
**Version:** 1.0.0

## 1. Import Required Libraries

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model persistence
import joblib
import json
import os

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")

## 2. Load and Explore Training Data

Since we don't have real historical data yet, we'll generate synthetic logistics data that mimics real-world delivery patterns.

In [ ]:
# Generate synthetic logistics data for ETA prediction
np.random.seed(42)
n_samples = 5000

# Indian cities with coordinates (lat, lng)
cities = {
    'Delhi': (28.6139, 77.2090),
    'Mumbai': (19.0760, 72.8777),
    'Bangalore': (12.9716, 77.5946),
    'Chennai': (13.0827, 80.2707),
    'Kolkata': (22.5726, 88.3639),
    'Hyderabad': (17.3850, 78.4867),
    'Pune': (18.5204, 73.8567),
    'Ahmedabad': (23.0225, 72.5714),
    'Jaipur': (26.9124, 75.7873),
    'Lucknow': (26.8467, 80.9462)
}

city_names = list(cities.keys())

# Generate data
data = {
    'shipment_id': [f'SHP{str(i).zfill(6)}' for i in range(n_samples)],
    'origin_city': np.random.choice(city_names, n_samples),
    'destination_city': np.random.choice(city_names, n_samples),
    'pickup_time': [datetime(2024, 1, 1) + timedelta(
        days=np.random.randint(0, 365),
        hours=np.random.randint(6, 22),
        minutes=np.random.randint(0, 60)
    ) for _ in range(n_samples)],
    'weight_kg': np.random.uniform(0.5, 500, n_samples),
    'volume_cbm': np.random.uniform(0.01, 5, n_samples),
    'vehicle_type': np.random.choice(['bike', 'van', 'truck', 'mini_truck'], n_samples, p=[0.3, 0.35, 0.2, 0.15]),
    'is_express': np.random.choice([0, 1], n_samples, p=[0.7, 0.3]),
    'is_fragile': np.random.choice([0, 1], n_samples, p=[0.85, 0.15]),
    'weather_score': np.random.uniform(0.5, 1.0, n_samples),  # 1 = clear, 0.5 = poor
    'traffic_factor': np.random.uniform(0.8, 2.5, n_samples),  # multiplier
    'num_stops': np.random.randint(0, 5, n_samples)
}

df = pd.DataFrame(data)

# Add coordinates
df['origin_lat'] = df['origin_city'].map(lambda x: cities[x][0])
df['origin_lng'] = df['origin_city'].map(lambda x: cities[x][1])
df['dest_lat'] = df['destination_city'].map(lambda x: cities[x][0])
df['dest_lng'] = df['destination_city'].map(lambda x: cities[x][1])

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()

In [ ]:
# Data overview
print("=" * 50)
print("DATA OVERVIEW")
print("=" * 50)
print(f"\nShape: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nBasic Statistics:")
df.describe()

## 3. Data Preprocessing and Cleaning

In [ ]:
# Check for missing values
print("Missing Values:")
print(df.isnull().sum())
print(f"\nTotal missing: {df.isnull().sum().sum()}")

# Remove same origin-destination (invalid shipments)
df = df[df['origin_city'] != df['destination_city']].copy()
print(f"\nAfter removing same origin-destination: {len(df)} rows")

# Validate coordinates
df = df[
    (df['origin_lat'].between(-90, 90)) & 
    (df['origin_lng'].between(-180, 180)) &
    (df['dest_lat'].between(-90, 90)) & 
    (df['dest_lng'].between(-180, 180))
].copy()

print(f"After coordinate validation: {len(df)} rows")

## 4. Feature Engineering

Create features that capture delivery time patterns:
- **Distance features**: Haversine distance between origin and destination
- **Time features**: Hour, day of week, weekend, rush hour
- **Categorical encoding**: Vehicle type, origin/destination zones

In [ ]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate haversine distance between two points in km"""
    R = 6371  # Earth's radius in km
    
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

# Calculate distance
df['distance_km'] = haversine_distance(
    df['origin_lat'], df['origin_lng'],
    df['dest_lat'], df['dest_lng']
)

# Time-based features
df['hour_of_day'] = df['pickup_time'].dt.hour
df['day_of_week'] = df['pickup_time'].dt.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['is_rush_hour'] = df['hour_of_day'].apply(
    lambda x: 1 if (8 <= x <= 10) or (17 <= x <= 20) else 0
)
df['month'] = df['pickup_time'].dt.month
df['is_night'] = df['hour_of_day'].apply(lambda x: 1 if x < 6 or x > 21 else 0)

# Encode categorical variables
le_vehicle = LabelEncoder()
le_origin = LabelEncoder()
le_dest = LabelEncoder()

df['vehicle_type_encoded'] = le_vehicle.fit_transform(df['vehicle_type'])
df['origin_zone_encoded'] = le_origin.fit_transform(df['origin_city'])
df['destination_zone_encoded'] = le_dest.fit_transform(df['destination_city'])

print("✅ Features created!")
print(f"\nNew features: {['distance_km', 'hour_of_day', 'day_of_week', 'is_weekend', 'is_rush_hour', 'month', 'is_night']}")

In [ ]:
# Generate realistic actual ETA (target variable)
# Base time = distance/average_speed + adjustments

def calculate_actual_eta(row):
    """Calculate realistic ETA in minutes based on features"""
    # Base speed varies by vehicle type (km/h)
    base_speeds = {'bike': 25, 'van': 45, 'mini_truck': 40, 'truck': 35}
    base_speed = base_speeds.get(row['vehicle_type'], 40)
    
    # Base time from distance
    base_time = (row['distance_km'] / base_speed) * 60  # minutes
    
    # Traffic factor adjustment
    base_time *= row['traffic_factor']
    
    # Rush hour adds 15-30% time
    if row['is_rush_hour']:
        base_time *= np.random.uniform(1.15, 1.30)
    
    # Weather impact
    base_time *= (2 - row['weather_score'])  # poor weather increases time
    
    # Night deliveries can be faster (less traffic)
    if row['is_night']:
        base_time *= 0.85
    
    # Each stop adds 10-20 minutes
    base_time += row['num_stops'] * np.random.uniform(10, 20)
    
    # Express shipments prioritized (faster)
    if row['is_express']:
        base_time *= 0.9
    
    # Add some random variation (±10%)
    base_time *= np.random.uniform(0.9, 1.1)
    
    return max(15, base_time)  # Minimum 15 minutes

df['actual_duration_minutes'] = df.apply(calculate_actual_eta, axis=1)

print(f"Target variable statistics:")
print(df['actual_duration_minutes'].describe())

## 5. Exploratory Data Analysis with Visualizations

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. ETA Distribution
axes[0, 0].hist(df['actual_duration_minutes'], bins=50, color='steelblue', edgecolor='white')
axes[0, 0].set_title('Distribution of Actual ETA', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Duration (minutes)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['actual_duration_minutes'].mean(), color='red', linestyle='--', label=f'Mean: {df["actual_duration_minutes"].mean():.0f} min')
axes[0, 0].legend()

# 2. ETA vs Distance
axes[0, 1].scatter(df['distance_km'], df['actual_duration_minutes'], alpha=0.3, c='steelblue', s=10)
axes[0, 1].set_title('ETA vs Distance', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Distance (km)')
axes[0, 1].set_ylabel('Duration (minutes)')

# 3. ETA by Hour of Day
hourly_avg = df.groupby('hour_of_day')['actual_duration_minutes'].mean()
axes[0, 2].bar(hourly_avg.index, hourly_avg.values, color='steelblue', edgecolor='white')
axes[0, 2].set_title('Average ETA by Hour of Day', fontsize=12, fontweight='bold')
axes[0, 2].set_xlabel('Hour')
axes[0, 2].set_ylabel('Avg Duration (minutes)')
axes[0, 2].axhline(df['actual_duration_minutes'].mean(), color='red', linestyle='--', alpha=0.7)

# 4. ETA by Day of Week
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
daily_data = df.groupby('day_of_week')['actual_duration_minutes'].agg(['mean', 'std'])
axes[1, 0].bar(range(7), daily_data['mean'], yerr=daily_data['std'], color='steelblue', edgecolor='white', capsize=3)
axes[1, 0].set_title('ETA by Day of Week', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Day')
axes[1, 0].set_ylabel('Avg Duration (minutes)')
axes[1, 0].set_xticks(range(7))
axes[1, 0].set_xticklabels(days)

# 5. ETA by Vehicle Type
vehicle_avg = df.groupby('vehicle_type')['actual_duration_minutes'].mean().sort_values()
axes[1, 1].barh(vehicle_avg.index, vehicle_avg.values, color='steelblue', edgecolor='white')
axes[1, 1].set_title('Average ETA by Vehicle Type', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Avg Duration (minutes)')

# 6. Distance Distribution
axes[1, 2].hist(df['distance_km'], bins=50, color='steelblue', edgecolor='white')
axes[1, 2].set_title('Distribution of Distance', fontsize=12, fontweight='bold')
axes[1, 2].set_xlabel('Distance (km)')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('../data/training/eta_eda_plots.png', dpi=150, bbox_inches='tight')
plt.show()

print("✅ EDA plots generated!")

In [ ]:
# Correlation Heatmap
numeric_cols = ['distance_km', 'weight_kg', 'volume_cbm', 'hour_of_day', 'day_of_week',
                'is_weekend', 'is_rush_hour', 'weather_score', 'traffic_factor',
                'num_stops', 'is_express', 'is_fragile', 'actual_duration_minutes']

corr_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, square=True, linewidths=0.5)
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nTop correlations with actual_duration_minutes:")
print(corr_matrix['actual_duration_minutes'].sort_values(ascending=False)[1:6])

## 6. Train-Test Split

In [ ]:
# Define features for model
FEATURE_COLUMNS = [
    'distance_km', 'weight_kg', 'volume_cbm', 'hour_of_day', 'day_of_week',
    'is_weekend', 'is_rush_hour', 'weather_score', 'traffic_factor',
    'vehicle_type_encoded', 'origin_zone_encoded', 'destination_zone_encoded',
    'num_stops', 'is_express', 'is_fragile', 'is_night', 'month'
]

TARGET_COLUMN = 'actual_duration_minutes'

X = df[FEATURE_COLUMNS].copy()
y = df[TARGET_COLUMN].copy()

# Split data: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set size: {len(X_train)} samples")
print(f"Test set size: {len(X_test)} samples")
print(f"\nFeatures used: {len(FEATURE_COLUMNS)}")
print(f"Target: {TARGET_COLUMN}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Data split and scaled!")

## 7. Model Training with Random Forest

In [ ]:
import time

# Initialize Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=15,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

# Train model
print("Training Random Forest model...")
start_time = time.time()

rf_model.fit(X_train_scaled, y_train)

training_time = time.time() - start_time
print(f"✅ Model trained in {training_time:.2f} seconds")

# Quick evaluation on training data
train_predictions = rf_model.predict(X_train_scaled)
train_mae = mean_absolute_error(y_train, train_predictions)
print(f"\nTraining MAE: {train_mae:.2f} minutes")

## 8. Hyperparameter Tuning

In [ ]:
# Define parameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

print("Running GridSearchCV for hyperparameter tuning...")
print(f"Parameter combinations to try: {3*3*3*3} = 81")

# Grid Search with 3-fold CV (reduced for speed)
grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid,
    cv=3,
    scoring='neg_mean_absolute_error',
    verbose=1,
    n_jobs=-1
)

start_time = time.time()
grid_search.fit(X_train_scaled, y_train)
tuning_time = time.time() - start_time

print(f"\n✅ Grid search completed in {tuning_time:.2f} seconds")
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV MAE: {-grid_search.best_score_:.2f} minutes")

# Use best model
best_model = grid_search.best_estimator_

## 9. Model Evaluation and Metrics

In [ ]:
# Predictions on test set
y_pred = best_model.predict(X_test_scaled)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("=" * 50)
print("MODEL EVALUATION RESULTS")
print("=" * 50)
print(f"\n📊 Test Set Metrics:")
print(f"   MAE  (Mean Absolute Error):    {mae:.2f} minutes")
print(f"   RMSE (Root Mean Squared Error): {rmse:.2f} minutes")
print(f"   R²   (R-squared):               {r2:.4f}")
print(f"   MAPE (Mean Absolute % Error):   {mape:.2f}%")

# Accuracy within thresholds
within_10 = np.mean(np.abs(y_test - y_pred) <= 10) * 100
within_20 = np.mean(np.abs(y_test - y_pred) <= 20) * 100
within_30 = np.mean(np.abs(y_test - y_pred) <= 30) * 100

print(f"\n🎯 Prediction Accuracy:")
print(f"   Within ±10 min: {within_10:.1f}%")
print(f"   Within ±20 min: {within_20:.1f}%")
print(f"   Within ±30 min: {within_30:.1f}%")

In [ ]:
# Visualization of predictions
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. Actual vs Predicted scatter plot
ax1 = axes[0, 0]
ax1.scatter(y_test, y_pred, alpha=0.5, edgecolors='none', s=50)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2, label='Perfect Prediction')
ax1.set_xlabel('Actual Duration (minutes)', fontsize=12)
ax1.set_ylabel('Predicted Duration (minutes)', fontsize=12)
ax1.set_title('Actual vs Predicted ETA', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Residual distribution
ax2 = axes[0, 1]
residuals = y_test - y_pred
ax2.hist(residuals, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax2.set_xlabel('Residual (Actual - Predicted)', fontsize=12)
ax2.set_ylabel('Frequency', fontsize=12)
ax2.set_title('Residual Distribution', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# 3. Residuals vs Predicted values
ax3 = axes[1, 0]
ax3.scatter(y_pred, residuals, alpha=0.5, edgecolors='none', s=50)
ax3.axhline(y=0, color='red', linestyle='--', linewidth=2)
ax3.set_xlabel('Predicted Duration (minutes)', fontsize=12)
ax3.set_ylabel('Residual', fontsize=12)
ax3.set_title('Residuals vs Predicted Values', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Error distribution by percentage
ax4 = axes[1, 1]
percentage_errors = np.abs((y_test - y_pred) / y_test) * 100
ax4.hist(percentage_errors, bins=50, edgecolor='black', alpha=0.7, color='coral')
ax4.axvline(x=10, color='green', linestyle='--', linewidth=2, label='10% threshold')
ax4.set_xlabel('Absolute Percentage Error (%)', fontsize=12)
ax4.set_ylabel('Frequency', fontsize=12)
ax4.set_title('Percentage Error Distribution', fontsize=14, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/eta_evaluation_plots.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📈 Evaluation plots saved to '../data/eta_evaluation_plots.png'")

## 10. Feature Importance Analysis

Understanding which features contribute most to ETA predictions.

In [ ]:
# Get feature importances
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("🔍 Top 15 Most Important Features for ETA Prediction:\n")
print(feature_importance.head(15).to_string(index=False))

# Visualize feature importance
fig, ax = plt.subplots(figsize=(12, 8))
top_features = feature_importance.head(15)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(top_features)))

bars = ax.barh(range(len(top_features)), top_features['importance'], color=colors)
ax.set_yticks(range(len(top_features)))
ax.set_yticklabels(top_features['feature'])
ax.invert_yaxis()
ax.set_xlabel('Feature Importance', fontsize=12)
ax.set_title('Top 15 Feature Importances for ETA Prediction', fontsize=14, fontweight='bold')
ax.grid(True, axis='x', alpha=0.3)

# Add value labels
for i, (bar, val) in enumerate(zip(bars, top_features['importance'])):
    ax.text(val + 0.005, bar.get_y() + bar.get_height()/2, f'{val:.3f}', 
            va='center', fontsize=10)

plt.tight_layout()
plt.savefig('../data/eta_feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Feature importance plot saved to '../data/eta_feature_importance.png'")

## 11. Save Trained Model

Export the trained model for production use.

In [ ]:
import json
from datetime import datetime
import os

# Create models directory if it doesn't exist
os.makedirs('../models', exist_ok=True)

# Save the trained model
model_path = '../models/eta_model_notebook.pkl'
joblib.dump(best_model, model_path)
print(f"✅ Model saved to: {model_path}")

# Save the scaler
scaler_path = '../models/eta_scaler_notebook.pkl'
joblib.dump(scaler, scaler_path)
print(f"✅ Scaler saved to: {scaler_path}")

# Save model metadata
metadata = {
    'model_type': 'RandomForestRegressor',
    'training_date': datetime.now().isoformat(),
    'features': feature_columns,
    'num_features': len(feature_columns),
    'training_samples': len(X_train),
    'test_samples': len(X_test),
    'best_params': best_params,
    'metrics': {
        'mae': float(mae),
        'rmse': float(rmse),
        'r2': float(r2),
        'mape': float(mape)
    },
    'accuracy_thresholds': {
        'within_10_min': float(within_10),
        'within_20_min': float(within_20),
        'within_30_min': float(within_30)
    }
}

metadata_path = '../models/eta_model_metadata.json'
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)
print(f"✅ Metadata saved to: {metadata_path}")

print("\n" + "=" * 50)
print("MODEL ARTIFACTS SUMMARY")
print("=" * 50)
print(f"📁 Model file:    {model_path}")
print(f"📁 Scaler file:   {scaler_path}")
print(f"📁 Metadata file: {metadata_path}")
print(f"\n🎯 Model Performance: R² = {r2:.4f}, MAE = {mae:.2f} min")

## 12. Conclusions and Next Steps

### Key Findings:
1. **Distance** is the most important predictor of ETA
2. **Time-based features** (rush hour, day of week) significantly impact predictions
3. **Traffic conditions** and **weather** play moderate roles
4. Model achieves good accuracy for logistics planning purposes

### Recommended Next Steps:
- Integrate real-time traffic API data for improved predictions
- Add historical traffic patterns by route
- Consider ensemble methods combining multiple algorithms
- Implement online learning for continuous model improvement
- A/B test predictions against actual delivery times

---
*Notebook completed - Model ready for production deployment*